# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770415799303                   -0.52    9.0    214ms
  2   -2.771692275371       -2.89       -1.32    1.0   94.4ms
  3   -2.771714417797       -4.65       -2.47    1.0   97.5ms
  4   -2.771714638090       -6.66       -3.15    1.0    100ms
  5   -2.771714713736       -7.12       -3.92    2.0    123ms
  6   -2.771714715073       -8.87       -4.51    1.0    110ms
  7   -2.771714715244       -9.77       -5.10    1.0    129ms
  8   -2.771714715248      -11.35       -5.66    1.0    171ms
  9   -2.771714715250      -11.92       -6.17    2.0    557ms
 10   -2.771714715250      -13.23       -6.75    1.0    147ms
 11   -2.771714715250   +  -14.51       -8.04    1.0    148ms


-0.0001345734991356166

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770494164993                   -0.52    9.0    273ms
  2   -2.771779270997       -2.89       -1.32    1.0    165ms
  3   -2.771801648909       -4.65       -2.45    1.0   94.8ms
  4   -2.771801994986       -6.46       -3.15    1.0    100ms
  5   -2.771802074170       -7.10       -4.16    2.0    118ms
  6   -2.771802074266      -10.02       -4.28    1.0    103ms
  7   -2.771802074470       -9.69       -5.21    1.0    108ms
  8   -2.771802074475      -11.33       -5.39    1.0    621ms
  9   -2.771802074476      -12.03       -5.91    1.0    106ms
 10   -2.771802074476      -12.92       -6.14    2.0    127ms
 11   -2.771802074476      -13.32       -6.45    1.0    132ms
 12   -2.771802074476      -13.74       -6.92    1.0    108ms
 13   -2.771802074476      -14.75       -7.59    1.0    123ms
 14   -2.771802074476   +  -14.18       -8.17    1.0    116ms


0.017612221673555452

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345734991356166
Displaced dipole:  0.017612221673555452
Polarizability :   1.7746795172691068


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920759516e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551294755e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852762134024e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694594472474e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088788677617e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.285835200023e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.067628154460e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.067628154460e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.286875058361e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 6.638841658490e-12
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.201555543174e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.201555543174e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.201369741543299e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.